# Indexing the new document.


- document is stored in the data folder
    1. Load the document 
    2. create embedding
    3. push to chroma db.


In [6]:
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain.vectorstores.chroma import Chroma

In [7]:
# loading the document.
pdf_loader_from_path = PyPDFDirectoryLoader("data")

In [8]:
loaded_pdf = pdf_loader_from_path.load()

In [9]:
loaded_pdf

[Document(metadata={'source': 'data\\ETS2_manual_en.pdf', 'page': 0}, page_content='MANUAL'),
 Document(metadata={'source': 'data\\ETS2_manual_en.pdf', 'page': 1}, page_content=''),
 Document(metadata={'source': 'data\\ETS2_manual_en.pdf', 'page': 2}, page_content='Welcome to Euro Truck Simulator 2\nThe Euro Truck Simulator 2 lets you experience the thrill of commanding the \nmost powerful machines that cruise the motorways. You will start as an enthu -\nsiast trucker for hire and will get the chance to build your own transportation empire spanning across the whole Europe. Configure your trucks from scratch, \nbuy as many as you can afford, explore every road and acquire a garage in ev -\nery city - these are just a few of the challenges that await. Godspeed, driver!  \nGetting Started\nSystem requirements\nMinimum System Requirements:\n Operating system: Windows XP/Vista/7\n Processor Dual core CPU 2.4 GHz\n 2 GB system memory\n Dedicated graphics card with 256 MB memory (GeForce 7600

In [10]:
# document is loaded as pages now we need to further split the pages into chucks to process

In [11]:
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

In [12]:
chunks = split_documents(loaded_pdf)

In [17]:
# embedding
from langchain_community.embeddings.ollama import OllamaEmbeddings


def get_embedding_function():
    embeddings = OllamaEmbeddings(model="llama3:8b")
    return embeddings

In [16]:
def add_to_chroma(chunks: list[Document]):
    # Load the existing database.
    db = Chroma(
        persist_directory="chroma", embedding_function=get_embedding_function()
    )

    # Calculate Page IDs.
    chunks_with_ids = calculate_chunk_ids(chunks)

    # Add or Update the documents.
    existing_items = db.get(include=[])  # IDs are always included by default
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    # Only add documents that don't exist in the DB.
    new_chunks = []
    for chunk in chunks_with_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)

    if len(new_chunks):
        print(f"👉 Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        db.persist()
    else:
        print("✅ No new documents to add")


def calculate_chunk_ids(chunks):

    # This will create IDs like "data/monopoly.pdf:6:2"
    # Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id

    return chunks

In [18]:
add_to_chroma(chunks)

c:\Users\nirva\Documents\GitHub\machine_Learning_Algorithms_python\genAI\RAG\rag_app\.venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


Number of existing documents in DB: 0
👉 Adding new documents: 18


c:\Users\nirva\Documents\GitHub\machine_Learning_Algorithms_python\genAI\RAG\rag_app\.venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [19]:
CHROMA_PATH = "chroma"

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

In [22]:
from langchain.vectorstores.chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama

In [23]:
def query_rag(query_text: str):
    # Prepare the DB.
    embedding_function = get_embedding_function()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_score(query_text, k=5)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    # print(prompt)

    model = Ollama(model="llama3:8b")
    response_text = model.invoke(prompt)

    sources = [doc.metadata.get("id", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)
    return response_text

In [24]:
query_rag("what is ETS2")

Response: Based on the provided manual, ETS2 refers to "Euro Truck Simulator 2", a video game that allows players to create their own profile, play as a truck driver, and navigate through different cities.
Sources: ['data\\ETS2_manual_en.pdf:0:0', 'data\\ETS2_manual_en.pdf:8:0', 'data\\ETS2_manual_en.pdf:4:0', 'data\\ETS2_manual_en.pdf:6:0', 'data\\ETS2_manual_en.pdf:11:0']


'Based on the provided manual, ETS2 refers to "Euro Truck Simulator 2", a video game that allows players to create their own profile, play as a truck driver, and navigate through different cities.'

In [25]:
query_rag("how can I get fly mode in ETS2")

Response: Unfortunately, there is no mention of a "fly mode" in the provided context. The options and menus mentioned are all related to creating profiles, switching between profiles, configuring controls, and quitting the game. There is no reference to flying or any kind of aerial gameplay in Euro Truck Simulator 2 (ETS2).
Sources: ['data\\ETS2_manual_en.pdf:4:0', 'data\\ETS2_manual_en.pdf:11:0', 'data\\ETS2_manual_en.pdf:9:2', 'data\\ETS2_manual_en.pdf:6:2', 'data\\ETS2_manual_en.pdf:8:0']


'Unfortunately, there is no mention of a "fly mode" in the provided context. The options and menus mentioned are all related to creating profiles, switching between profiles, configuring controls, and quitting the game. There is no reference to flying or any kind of aerial gameplay in Euro Truck Simulator 2 (ETS2).'

In [26]:
query_rag("how to change time in ETS2")

Response: The answer cannot be found in the provided context. The text only discusses system requirements, installation process, and activation of Euro Truck Simulator 2 (ETS2), but does not mention how to change time in the game.
Sources: ['data\\ETS2_manual_en.pdf:11:0', 'data\\ETS2_manual_en.pdf:9:2', 'data\\ETS2_manual_en.pdf:6:2', 'data\\ETS2_manual_en.pdf:2:1', 'data\\ETS2_manual_en.pdf:3:1']


'The answer cannot be found in the provided context. The text only discusses system requirements, installation process, and activation of Euro Truck Simulator 2 (ETS2), but does not mention how to change time in the game.'

In [27]:
query_rag("how do I break in ETS2")

Response: Based on the provided manual, to start playing Euro Truck Simulator 2 (ETS2), you can follow these steps:

1. Create a profile by choosing your name, gender, picture, preferred truck design, and company logo and name.
2. Confirm your profile creation and start playing.

Alternatively, you can also go directly into the world by pressing the "Drive" button from the Main Menu (option 1). This will allow you to drive wherever you please, providing you have a truck or are on a job.

Note that this is assuming you have already installed the game.
Sources: ['data\\ETS2_manual_en.pdf:0:0', 'data\\ETS2_manual_en.pdf:4:0', 'data\\ETS2_manual_en.pdf:6:0', 'data\\ETS2_manual_en.pdf:8:0', 'data\\ETS2_manual_en.pdf:11:0']


'Based on the provided manual, to start playing Euro Truck Simulator 2 (ETS2), you can follow these steps:\n\n1. Create a profile by choosing your name, gender, picture, preferred truck design, and company logo and name.\n2. Confirm your profile creation and start playing.\n\nAlternatively, you can also go directly into the world by pressing the "Drive" button from the Main Menu (option 1). This will allow you to drive wherever you please, providing you have a truck or are on a job.\n\nNote that this is assuming you have already installed the game.'

In [28]:
query_rag("what is the control to apply break in ETS2")

Response: Based on the provided manual, there is no specific mention of how to apply the brakes in Euro Truck Simulator 2 (ETS2). The "Control Keys" section does exist, but it only provides information on reassigning keys through the "Options" -> "Keyboard" menu. It doesn't specify which key controls the brakes.

If you're looking for information on applying the brakes, I recommend consulting the game's manual or online documentation for more specific guidance.
Sources: ['data\\ETS2_manual_en.pdf:0:0', 'data\\ETS2_manual_en.pdf:4:0', 'data\\ETS2_manual_en.pdf:11:0', 'data\\ETS2_manual_en.pdf:8:0', 'data\\ETS2_manual_en.pdf:9:2']


'Based on the provided manual, there is no specific mention of how to apply the brakes in Euro Truck Simulator 2 (ETS2). The "Control Keys" section does exist, but it only provides information on reassigning keys through the "Options" -> "Keyboard" menu. It doesn\'t specify which key controls the brakes.\n\nIf you\'re looking for information on applying the brakes, I recommend consulting the game\'s manual or online documentation for more specific guidance.'

In [29]:
query_rag("Summerize  Main Vehicle Controls in ETS2")

Response: Unfortunately, there are no "Main Vehicle Controls" described in the provided context. The only controls mentioned are:

* Cameras Controls
* Miscellaneous Controls

No information is given about main vehicle controls or any specific control options for controlling vehicles in Euro Truck Simulator 2.
Sources: ['data\\ETS2_manual_en.pdf:9:2', 'data\\ETS2_manual_en.pdf:11:0', 'data\\ETS2_manual_en.pdf:2:1', 'data\\ETS2_manual_en.pdf:6:2', 'data\\ETS2_manual_en.pdf:3:1']


'Unfortunately, there are no "Main Vehicle Controls" described in the provided context. The only controls mentioned are:\n\n* Cameras Controls\n* Miscellaneous Controls\n\nNo information is given about main vehicle controls or any specific control options for controlling vehicles in Euro Truck Simulator 2.'

In [30]:

query_rag("Summerize  Cameras Controls in ETS2")

Response: Based on the provided context, there is no specific information about "Cameras Controls" in Euro Truck Simulator 2 (ETS2). The context only provides general information about installing and playing the game, as well as some basic controls and features. It does not mention any camera-specific settings or controls.
Sources: ['data\\ETS2_manual_en.pdf:9:2', 'data\\ETS2_manual_en.pdf:11:0', 'data\\ETS2_manual_en.pdf:2:1', 'data\\ETS2_manual_en.pdf:6:2', 'data\\ETS2_manual_en.pdf:3:1']


'Based on the provided context, there is no specific information about "Cameras Controls" in Euro Truck Simulator 2 (ETS2). The context only provides general information about installing and playing the game, as well as some basic controls and features. It does not mention any camera-specific settings or controls.'

In [31]:
query_rag("how to install ets")

Response: Based on the provided manual, there is no information about installing Euro Truck Simulator 2 (ETS). The manual appears to be a guide for already installed game settings and controls. To install ETS, you would need to follow the installation instructions provided by the game's developers or publisher, which are not included in this specific manual.
Sources: ['data\\ETS2_manual_en.pdf:0:0', 'data\\ETS2_manual_en.pdf:6:0', 'data\\ETS2_manual_en.pdf:8:0', 'data\\ETS2_manual_en.pdf:4:0', 'data\\ETS2_manual_en.pdf:9:1']


"Based on the provided manual, there is no information about installing Euro Truck Simulator 2 (ETS). The manual appears to be a guide for already installed game settings and controls. To install ETS, you would need to follow the installation instructions provided by the game's developers or publisher, which are not included in this specific manual."